In [10]:
# ============================================================
# Track B FINAL (one-cell runnable code with demo example)
# - B* (적정보증금) 범위: mu 보정 전/후 비교
# - 시나리오 민감도: base / -10% / -20%
# ============================================================

import numpy as np
import pandas as pd
from math import erf, sqrt

# ---------------------------
# 1) 파라미터 정의 (너희 최종값으로 고정)
# ---------------------------
MU_HAT       = -0.007366   # 보정 전 mu (3년 평균 등)
MU_ANNUAL    = -0.002734   # 보정 후 mu (금리 보정 반영)
SIGMA_ANNUAL = 0.3133634101269932

ALPHA_MEAN = 0.754475
ALPHA_P10  = 0.587731
ALPHA_MODE = "p10"                # "mean" or "p10"
ALPHA_USED = ALPHA_P10 if ALPHA_MODE == "p10" else ALPHA_MEAN

SCENARIOS = {
    "base": 0.00,
    "stress10": -0.10,
    "stress20": -0.20,
}

# 정책 파라미터(예시): "평균 손실 EL이 5천만원 넘으면 위험" 같은 기준
EL_CAP = 50000
SCENARIO_FOR_BSTAR = "base"

# ---------------------------
# 2) 표준정규 CDF (scipy 없이)
# ---------------------------
def norm_cdf(x: np.ndarray) -> np.ndarray:
    return 0.5 * (1.0 + np.vectorize(erf)(x / sqrt(2.0)))

# ---------------------------
# 3) PD: GBM 폐형식
# ---------------------------
def pd_gbm_closed_form(V0, B, T, mu, sigma):
    """
    PD = P(V_T < B)
       = Φ( (ln(B/V0) - (mu - 0.5*sigma^2)T) / (sigma*sqrt(T)) )
    """
    V0 = np.asarray(V0, dtype=float)
    B  = np.asarray(B, dtype=float)
    T  = np.asarray(T, dtype=float)

    out = np.full_like(V0, np.nan, dtype=float)
    valid = (V0 > 0) & (B > 0) & (T > 0) & np.isfinite(V0) & np.isfinite(B) & np.isfinite(T)
    if valid.sum() == 0:
        return out

    # sigma<=0: 확정적 비교
    if sigma <= 0:
        VT_det = V0[valid] * np.exp(mu * T[valid])
        out[valid] = (VT_det < B[valid]).astype(float)
        return out

    z = (np.log(B[valid] / V0[valid]) - (mu - 0.5 * sigma**2) * T[valid]) / (sigma * np.sqrt(T[valid]))
    out[valid] = norm_cdf(z)
    return out

# ---------------------------
# 4) EL: E[(B - alpha*V_T)^+] 폐형식
# ---------------------------
def expected_loss_closed_form(V0, B, T, mu, sigma, alpha):
    """
    Loss = max(B - alpha*V_T, 0)
    Return E[Loss] under GBM (원화 기준 기대손실액)
    """
    V0 = np.asarray(V0, dtype=float)
    B  = np.asarray(B, dtype=float)
    T  = np.asarray(T, dtype=float)

    out = np.full_like(V0, np.nan, dtype=float)
    valid = (V0 > 0) & (B > 0) & (T > 0) & np.isfinite(V0) & np.isfinite(B) & np.isfinite(T)
    if valid.sum() == 0:
        return out

    # sigma<=0: 확정적 케이스 (0으로 나누기 방지)
    if sigma <= 0:
        VT_det = V0[valid] * np.exp(mu * T[valid])
        loss_det = np.maximum(B[valid] - alpha * VT_det, 0.0)
        out[valid] = loss_det
        return out

    m = np.log(V0[valid]) + (mu - 0.5 * sigma**2) * T[valid]
    s = sigma * np.sqrt(T[valid])

    # 손실 발생 조건: alpha*V_T < B  <=>  V_T < B/alpha
    K = B[valid] / alpha
    d = (np.log(K) - m) / s

    Phi_d = norm_cdf(d)
    term2 = np.exp(m + 0.5 * s**2) * norm_cdf(d - s)

    # E[(B - alpha V_T)^+] = B*Φ(d) - alpha*E[V_T 1(V_T<K)]
    out[valid] = B[valid] * Phi_d - alpha * term2
    out[valid] = np.maximum(out[valid], 0.0)  # 수치 오차 방지
    return out

# ---------------------------
# 5) 메인: df에 PD/LGD/EL 컬럼 추가
# ---------------------------
def add_trackB_risk_columns(
    df: pd.DataFrame,
    v0_col: str = "hedonic_price",
    b_col: str  = "deposit",
    t_col: str  = "term",
    mu: float = MU_ANNUAL,
    sigma: float = SIGMA_ANNUAL,
    alpha: float = ALPHA_USED,
    scenarios: dict = SCENARIOS
) -> pd.DataFrame:

    out = df.copy()

    # --- 필수 컬럼 체크
    for col in [v0_col, b_col, t_col]:
        if col not in out.columns:
            raise ValueError(f"필수 컬럼 누락: {col}")

    # --- 숫자화
    V0 = pd.to_numeric(out[v0_col], errors="coerce").to_numpy(dtype=float)
    B  = pd.to_numeric(out[b_col],  errors="coerce").to_numpy(dtype=float)
    out[t_col] = pd.to_numeric(out[t_col], errors="coerce")

    # --- T 유효성 체크
    if out[t_col].isna().any():
        raise ValueError("계약기간(T)에 NaN 값이 존재합니다. 입력값을 확인하세요.")
    if (out[t_col] <= 0).any():
        raise ValueError("계약기간(T)은 양수여야 합니다. 입력값을 확인하세요.")

    out["T_years"] = out[t_col]
    T = out["T_years"].to_numpy(dtype=float)

    # --- 전세가율
    out["jeonse_ratio"] = np.where((V0 > 0) & np.isfinite(V0) & np.isfinite(B), B / V0, np.nan)

    for s_name, shock in scenarios.items():
        V0_s = V0 * (1.0 + shock)
        V0_s = np.where(V0_s > 0, V0_s, np.nan)

        # PD / EL (폐형식)
        out[f"PD_{s_name}"] = pd_gbm_closed_form(V0_s, B, T, mu, sigma)
        out[f"EL_{s_name}"] = expected_loss_closed_form(V0_s, B, T, mu, sigma, alpha)

        # LGD_cond = EL / PD
        PD_arr = out[f"PD_{s_name}"].to_numpy(dtype=float)
        EL_arr = out[f"EL_{s_name}"].to_numpy(dtype=float)

        out[f"LGD_{s_name}"] = np.where(
            (PD_arr > 1e-12) & np.isfinite(PD_arr) & np.isfinite(EL_arr),
            EL_arr / PD_arr,
            0.0
        )

    return out

# ---------------------------
# 6) B* (권장 보증금 상한) 역산
# ---------------------------
def EL_of_B_closed_form(B, V0, T, mu, sigma, alpha, shock):
    V0_s = V0 * (1.0 + shock)
    if (not np.isfinite(V0_s)) or (V0_s <= 0) or (not np.isfinite(B)) or (B <= 0) or (T <= 0):
        return np.nan
    return float(expected_loss_closed_form(np.array([V0_s]), np.array([B]), np.array([T]), mu, sigma, alpha)[0])

def find_B_star_by_EL_closed_form(
    V0, T, mu, sigma, alpha, shock,
    EL_CAP,
    B_low=1.0,
    B_high_init=None,
    tol=100.0,
    max_iter=80
):
    V0 = float(V0); T = float(T); EL_CAP = float(EL_CAP)
    if V0 <= 0 or T <= 0 or EL_CAP < 0:
        return np.nan

    B_high = max(1.0, V0) if B_high_init is None else float(B_high_init)

    el_low = EL_of_B_closed_form(B_low, V0, T, mu, sigma, alpha, shock)
    if not np.isfinite(el_low):
        return np.nan
    if el_low > EL_CAP:
        return B_low

    el_high = EL_of_B_closed_form(B_high, V0, T, mu, sigma, alpha, shock)
    expand = 0
    while np.isfinite(el_high) and el_high <= EL_CAP and expand < 25:
        B_high *= 1.5
        el_high = EL_of_B_closed_form(B_high, V0, T, mu, sigma, alpha, shock)
        expand += 1

    if (not np.isfinite(el_high)) or (el_high <= EL_CAP):
        return B_high

    low, high = B_low, B_high
    for _ in range(max_iter):
        mid = 0.5 * (low + high)
        el_mid = EL_of_B_closed_form(mid, V0, T, mu, sigma, alpha, shock)

        if (not np.isfinite(el_mid)) or (el_mid > EL_CAP):
            high = mid
        else:
            low = mid

        if (high - low) <= tol:
            break

    return low

def B_star_range_two_mu(V0, T, sigma, alpha, shock, EL_CAP, mu_before, mu_after, tol=100.0):
    b1 = find_B_star_by_EL_closed_form(V0, T, mu_before, sigma, alpha, shock, EL_CAP, tol=tol)
    b2 = find_B_star_by_EL_closed_form(V0, T, mu_after,  sigma, alpha, shock, EL_CAP, tol=tol)
    return b1, b2

# ---------------------------
# 7) 시나리오 민감도 리포트
# ---------------------------
def scenario_sensitivity_report(df_out: pd.DataFrame, idx: int = 0, make_plot: bool = True):
    row = df_out.iloc[idx]

    scenarios = ["base", "stress10", "stress20"]
    labels    = ["정상(0%)", "-10% 하락", "-20% 하락"]

    records = []
    for s, lab in zip(scenarios, labels):
        records.append({
            "scenario": lab,
            "PD": float(row.get(f"PD_{s}", np.nan)),
            "LGD_cond": float(row.get(f"LGD_{s}", np.nan)),
            "EL": float(row.get(f"EL_{s}", np.nan)),
        })

    rep = pd.DataFrame(records)

    base_el = rep.loc[rep["scenario"] == "정상(0%)", "EL"].iloc[0]
    el_20   = rep.loc[rep["scenario"] == "-20% 하락", "EL"].iloc[0]

    rep["ΔEL_vs_base"] = rep["EL"] - base_el
    rep["EL_ratio_vs_base"] = rep["EL"] / (base_el if base_el > 0 else np.nan)
    slope_per_1pct = (el_20 - base_el) / 20.0

    return rep, base_el, el_20, slope_per_1pct

=== 입력값 ===
   hedonic_price  전월세_평균_보증금(만원)  계약기간(년)
0         190000          150000      2.0

=== Track B 출력(PD/LGD/EL) ===
   hedonic_price  전월세_평균_보증금(만원)  계약기간(년)  jeonse_ratio   PD_base  \
0         190000          150000      2.0      0.789474  0.382282   

        LGD_base       EL_base  PD_stress10   LGD_stress10   EL_stress10  \
0  123737.687785  47302.685395     0.475382  116240.427528  55258.660163   

   PD_stress20   LGD_stress20   EL_stress20  
0     0.580836  110328.521535  64082.751018  

=== 적정보증금(B*) 범위 분석 ===
[기준 시나리오]        : base
[손실 기준(EL_CAP)]    : 50,000 만원
[현재 보증금]          : 150,000.0 만원

▶ μ 보정 전 기준
   - 적정보증금 B*     : 152,334.1826171875 만원
   - 여유/초과 금액    : 2,334.1826171875 만원

▶ μ 보정 후 기준 (금리 반영)
   - 적정보증금 B*     : 153,261.912109375 만원
   - 여유/초과 금액    : 3,261.912109375 만원

[요약]
현재 보증금은 금리 반영 기준에서도 감내 가능한 범위입니다.

=== 시나리오 민감도 리포트 ===

[요약 민감도]
- base EL          : 47,302.68539485127
- -20% EL          : 64,082.75101761082
- ΔEL(-20 - base)  : 16,780.06

,hedonic_price,전월세_평균_보증금(만원),계약기간(년),T_years,jeonse_ratio,PD_base,EL_base,LGD_base,PD_stress10,EL_stress10,LGD_stress10,PD_stress20,EL_stress20,LGD_stress20
0,190000,150000,2.0,2.0,0.789474,0.382282,47302.685395,123737.687785,0.475382,55258.660163,116240.427528,0.580836,64082.751018,110328.521535
